In [ ]:
!pip uninstall -y typing-extensions typing-inspection starlette pydantic grpcio fastapi ipython torch torchvision torchaudio

In [ ]:
!python -m pip install --upgrade pip setuptools wheel
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu121

In [ ]:
!git clone https://github.com/ace-step/ACE-Step.git
%cd ACE-Step

In [ ]:
!pip install -r requirements.txt
!pip install -e .

In [ ]:
import sys
import os

repo_path = os.getenv('ACE_STEP_REPO_PATH', os.getcwd())
os.chdir(repo_path)

if repo_path not in sys.path:
    sys.path.append(repo_path)

print("Working directory:", os.getcwd())
print("In sys.path:", repo_path in sys.path)

In [ ]:
import os
from huggingface_hub import snapshot_download

repo_id = "ACE-Step/ACE-Step-v1-3.5B"
cache_root = os.getenv('ACE_STEP_WEIGHTS_PATH', os.path.join(repo_path, '..', 'acestep_checkpoints'))
os.makedirs(cache_root, exist_ok=True)

snapshot_dir = snapshot_download(repo_id=repo_id, cache_dir=cache_root)
print("Snapshot directory:", snapshot_dir)

In [ ]:
import torch
from acestep.pipeline_ace_step import ACEStepPipeline

pipe = ACEStepPipeline(
    checkpoint_dir=snapshot_dir,
    dtype="bfloat16",      
    device_id=0,           
    torch_compile=False
)

pipe.load_checkpoint(checkpoint_dir=snapshot_dir)
print("Pipeline loaded.")

In [ ]:
def generate_music(text_prompt: str, duration: int = 30):
    waveform = model.generate(text=text_prompt, duration=duration)
    return waveform

In [ ]:
duration = 30
prompt = ""
lyrics = ""  # leave empty if not using lyrics

audio_path = os.path.join(os.path.dirname(snapshot_dir), "generated_music.wav")

pipe(
    audio_duration=duration,
    prompt=prompt,
    lyrics=lyrics,
    infer_step=60,                 # 40–80 are typical
    guidance_scale=12.0,           # 8–16 are typical
    scheduler_type="euler",        # or "heun"
    cfg_type="apg",                # or "cfg", "cfg_star"
    omega_scale=10.0,
    manual_seeds="",
    guidance_interval=0.5,
    guidance_interval_decay=0.0,
    min_guidance_scale=3.0,
    use_erg_tag=True,
    use_erg_lyric=True,
    use_erg_diffusion=True,
    oss_steps="",
    guidance_scale_text=0.0,
    guidance_scale_lyric=0.0,
    save_path=audio_path
)

print("Saved to:", audio_path)